In [7]:
# Description 

# to explore, segment, and cluster the neighborhoods in the city of Toronto. 

<h1> Segmentation and Clustering of the city of Toronto Neighborhoods

In [8]:
# needed Librairies: 
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

<h3> 1.Importing data from the wikipedia website for the city of Toronto neighborhoods 

In [15]:
import requests 
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [16]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

<h3> 2. Extracting a table from the Wikipedia website using the soup tools 

In [17]:
my_table = soup.find('table',{'class':'wikitable sortable'})


In this step, we scan through the table created above in order to eliminate the HTML marker and distill the information of the postal codes, boroughs and neighborhoods

In [18]:



    # preinit list of lists
    rows = my_table.findAll('tr')
    row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
    ncols = max(row_lengths)
    nrows = len(rows)
    data = []
    for i in range(nrows):
        rowD = []
        for j in range(ncols):
            rowD.append('')
        data.append(rowD)

    # process html
    for i in range(len(rows)):
        row = rows[i]
        rowD = []
        cells = row.findAll(["td", "th"])
        for j in range(len(cells)):
            cell = cells[j]

            #lots of cells span cols and rows so lets deal with that
            cspan = int(cell.get('colspan', 1))
            rspan = int(cell.get('rowspan', 1))
            l = 0
            for k in range(rspan):
                # Shifts to the first empty cell of this row
                while data[i + k][j + l]:
                    l += 1
                for m in range(cspan):
                    cell_n = j + l + m
                    row_n = i + k
                    # in some cases the colspan can overflow the table, in those cases just get the last item
                    cell_n = min(cell_n, len(data[row_n])-1)
                    data[row_n][cell_n] += cell.text
                    print(cell.text)

        data.append(rowD)

# write data out to tab seperated format
    page = os.path.split(website_url)[1]
    fname = 'output_{}_t{}.tsv'.format(page, tn)
    f = codecs.open(fname, 'w')
    for i in range(nrows):
        rowStr = '\t'.join(data[i])
        rowStr = rowStr.replace('\n', '')
        print(rowStr)
        f.write(rowStr + '\n')

    f.close()

Postal code

Borough

Neighborhood

M1A

Not assigned



M2A

Not assigned



M3A

North York

Parkwoods

M4A

North York

Victoria Village

M5A

Downtown Toronto

Regent Park / Harbourfront

M6A

North York

Lawrence Manor / Lawrence Heights

M7A

Downtown Toronto

Queen's Park / Ontario Provincial Government

M8A

Not assigned



M9A

Etobicoke

Islington Avenue

M1B

Scarborough

Malvern / Rouge

M2B

Not assigned



M3B

North York

Don Mills

M4B

East York

Parkview Hill / Woodbine Gardens

M5B

Downtown Toronto

Garden District, Ryerson

M6B

North York

Glencairn

M7B

Not assigned



M8B

Not assigned



M9B

Etobicoke

West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale

M1C

Scarborough

Rouge Hill / Port Union / Highland Creek

M2C

Not assigned



M3C

North York

Don Mills

M4C

East York

Woodbine Heights

M5C

Downtown Toronto

St. James Town

M6C

York

Humewood-Cedarvale

M7C

Not assigned



M8C

Not assigned



M9C

Etobicoke

Eringate / Bloor

NameError: name 'os' is not defined

<h3> 3. Converting the table into a workable DataFrame using Pandas 

In [56]:
# define the dataframe columns
df=pd.DataFrame(data)
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header


                 

df['Postal code'] = df['Postal code\n'].str.replace(r'\b\n$', '', regex=True).str.strip()
df['Borough'] = df['Borough\n'].str.replace(r'\b\n$', '', regex=True).str.strip()
df['Neighborhood'] = df['Neighborhood\n'].str.replace(r'\b\n$', '', regex=True).str.strip()
df.drop(['Postal code\n', 'Borough\n','Neighborhood\n'], inplace = True, axis=1)
df.columns
# Drop rows with any empty cells
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace= True)
df

,Postal code,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M8A,Not assigned,
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Malvern / Rouge


In [57]:
#making sure that all the postal codes that do not have a corresponding borough are dropped
df = df[df['Borough'] != 'Not assigned']
df['Neighborhood'].replace('/',',',regex= True, inplace= True)

<h3> Here is the final Dataframe that has all the Neighborhoods and the Boroughs of the city of Toronto

In [58]:

df

,Postal code,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park , Harbourfront"
6,M6A,North York,"Lawrence Manor , Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,"Malvern , Rouge"
12,M3B,North York,Don Mills
13,M4B,East York,"Parkview Hill , Woodbine Gardens"
14,M5B,Downtown Toronto,"Garden District, Ryerson"


<h3> Here is the number of rows and columns of the final dataframe

In [59]:
df.shape


(103, 3)

<h3> 4. Adding Longitude and Latitude coordinates to the postal codes 

In [60]:
import pandas as pd
geo_data = pd.read_csv("http://cocl.us/Geospatial_data")
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [61]:

# Drop rows with any empty cells
geo_data.dropna(axis=0, how='any', thresh=None, subset=None, inplace= True)
geo_data.shape


(103, 3)

In [65]:
# making sure that our table is organized by postal code in ascending order
df.sort_values(['Postal code'], inplace= True)
df=df.reset_index(drop=True)
df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [66]:
# inserting the Latitude and Longitude to the df dataframe
df['Latitude']=geo_data['Latitude']
df['Longitude']=geo_data['Longitude']
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
